In [77]:
from nltk.util import ngrams
from nltk.util import bigrams
import pandas as pd
import numpy as np
import re
import operator

In [78]:
# reading ip data 
df = pd.read_excel("Merged_Apollo_Designations.xlsx")
print(len(df))
df.head()

25445


,Apollo Function,Apollo Division,Title
0,Consulting,Consultant,"Best Selling Author, Keynote Speaker and Strat..."
1,NaN,NaN,SAP S/4HANA Finance Managing Consultant
2,NaN,NaN,"Management Consultant, Author, Professional Sp..."
3,NaN,NaN,Consulting Professor
4,NaN,NaN,CEO / CMO / Strategist / Keynote Speaker / Con...


In [79]:
df['Title'] = df['Title'].replace(' ', np.nan)
df = df.dropna(axis=0, subset=['Title'])
print(len(df))

24619


In [80]:
df=df.ffill(axis = 0)
df.head()

,Apollo Function,Apollo Division,Title
0,Consulting,Consultant,"Best Selling Author, Keynote Speaker and Strat..."
1,Consulting,Consultant,SAP S/4HANA Finance Managing Consultant
2,Consulting,Consultant,"Management Consultant, Author, Professional Sp..."
3,Consulting,Consultant,Consulting Professor
4,Consulting,Consultant,CEO / CMO / Strategist / Keynote Speaker / Con...


In [81]:
def split_in_csv(word, no):
    word = str(word)
    k = word.split()
    n_gram = list(ngrams(k, n=no))
    return n_gram

def get_uni_gram(word):
    word = str(word)
    k = word.split()
    return k

In [82]:
def clean_text(text):
    text=str(text)
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    BAD_SYMBOLS_RE = re.compile('[^0-9A-Za-z #+_]')
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
#     text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    return text

In [83]:
target_col_name='Title'
df[target_col_name] = df.apply(lambda x: clean_text(x[target_col_name]), axis = 1) ########

df['uni-gram'] = df.apply(lambda x: get_uni_gram(x[target_col_name]), axis = 1)
df['bi-gram'] = df.apply(lambda x: split_in_csv(x[target_col_name], 2), axis = 1)
df['tri-gram'] = df.apply(lambda x: split_in_csv(x[target_col_name], 3), axis = 1)

In [84]:
df.head()

,Apollo Function,Apollo Division,Title,uni-gram,bi-gram,tri-gram
0,Consulting,Consultant,Best Selling Author Keynote Speaker and Strat...,"[Best, Selling, Author, Keynote, Speaker, and,...","[(Best, Selling), (Selling, Author), (Author, ...","[(Best, Selling, Author), (Selling, Author, Ke..."
1,Consulting,Consultant,SAP S 4HANA Finance Managing Consultant,"[SAP, S, 4HANA, Finance, Managing, Consultant]","[(SAP, S), (S, 4HANA), (4HANA, Finance), (Fina...","[(SAP, S, 4HANA), (S, 4HANA, Finance), (4HANA,..."
2,Consulting,Consultant,Management Consultant Author Professional Sp...,"[Management, Consultant, Author, Professional,...","[(Management, Consultant), (Consultant, Author...","[(Management, Consultant, Author), (Consultant..."
3,Consulting,Consultant,Consulting Professor,"[Consulting, Professor]","[(Consulting, Professor)]",[]
4,Consulting,Consultant,CEO CMO Strategist Keynote Speaker Con...,"[CEO, CMO, Strategist, Keynote, Speaker, Consu...","[(CEO, CMO), (CMO, Strategist), (Strategist, K...","[(CEO, CMO, Strategist), (CMO, Strategist, Key..."


In [85]:
fn_group=df.groupby('Apollo Function')
list_fn_groups = df['Apollo Function'].unique().tolist()

In [86]:
count=0
for i in list_fn_groups:
    temp=fn_group.get_group(i)
    print("****",i,"****")
    count+=len(temp['Apollo Division'].unique().tolist())
print(count)

**** Consulting ****
**** Sales ****
**** Operations ****
**** Medical & Health ****
**** Marketing ****
**** Education ****
**** Finance ****
**** Human Resources ****
**** Information Technology ****
**** Legal ****
192


In [87]:
def get_dicts(df):
    dict_uni={}
    for i in df['uni-gram']:
        for j in i:
            if j not in dict_uni:
                dict_uni[j]=1
            else:
                dict_uni[j]+=1
    dict_uni = dict(sorted(  dict_uni.items(),
                            key=operator.itemgetter(1),
                            reverse=True))
    dict_bi={}
    for i in df['bi-gram']:
        i=list(map(" ".join, i))
        for j in i:
            if j not in dict_bi:
                dict_bi[j]=1
            else:
                dict_bi[j]+=1
    dict_bi = dict(sorted(  dict_bi.items(),
                            key=operator.itemgetter(1),
                            reverse=True))
    dict_tri={}
    for i in df['tri-gram']:
        i=list(map(" ".join, i))
        for j in i:
            if j not in dict_tri:
                dict_tri[j]=1
            else:
                dict_tri[j]+=1
    dict_tri = dict(sorted(  dict_tri.items(),
                            key=operator.itemgetter(1),
                            reverse=True))
    return dict_uni,dict_bi,dict_tri

In [88]:
for i in list_fn_groups:
    temp=fn_group.get_group(i)
    list_div_groups=temp['Apollo Division'].unique().tolist()
    for j in list_div_groups:
        list_Temp=temp.loc[temp['Apollo Division']==j]
        print("*",i,"*",j,"-->",len(list_Temp))

* Consulting * Consultant --> 125
* Sales * Account Management --> 125
* Sales * Business Development --> 125
* Sales * Channel Sales --> 125
* Sales * Customer Retention & Development --> 125
* Sales * Customer Success --> 100
* Sales * Field / Outside Sales --> 125
* Sales * Inside Sales --> 125
* Sales * Partnerships --> 125
* Sales * Revenue Operations --> 125
* Sales * Sales --> 125
* Sales * Sales Enablement --> 125
* Sales * Sales Engineering --> 125
* Sales * Sales Operations --> 125
* Sales * Sales Training --> 125
* Operations * Call Center --> 125
* Operations * Construction --> 125
* Operations * Corporate Strategy --> 125
* Operations * Customer Service / Support --> 125
* Operations * Enterprise Resource Planning --> 125
* Operations * Facilities Management --> 119
* Operations * Leasing --> 125
* Operations * Logistics --> 125
* Operations * Office Operations --> 125
* Operations * Operations --> 125
* Operations * Physical Securirty --> 125
* Operations * Project Develo

In [89]:
list_fn_groups

['Consulting',
 'Sales',
 'Operations',
 'Medical & Health',
 'Marketing',
 'Education',
 'Finance',
 'Human Resources',
 'Information Technology',
 'Legal']

In [90]:
for i in list_fn_groups:
    temp=fn_group.get_group(i)
    list_div_groups=temp['Apollo Division'].unique().tolist()
    for j in list_div_groups:
        list_Temp=temp.loc[temp['Apollo Division']==j]
        if(len(list_Temp)>150):
            print("*",i,"*",j,"-->",len(list_Temp))

* Marketing * Digital Marketing --> 249
* Marketing * Artificial Intelligence / Machine Learning --> 250
* Information Technology * Quality Assurance --> 250
* Information Technology * Database Administration --> 264
* Information Technology * IT Strategy --> 250
* Information Technology * Storage & Disaster Recovery --> 250
* Legal * Acquisitions --> 249


In [91]:
temp.loc[temp['Apollo Division'] == 'Privacy']

,Apollo Function,Apollo Division,Title,uni-gram,bi-gram,tri-gram
20336,Legal,Privacy,Privacy Program Manager,"[Privacy, Program, Manager]","[(Privacy, Program), (Program, Manager)]","[(Privacy, Program, Manager)]"
20337,Legal,Privacy,Certification Committee Consultant: IOT AI S...,"[Certification, Committee, Consultant:, IOT, A...","[(Certification, Committee), (Committee, Consu...","[(Certification, Committee, Consultant:), (Com..."
20338,Legal,Privacy,Director of Engineering Android Security Pri...,"[Director, of, Engineering, Android, Security,...","[(Director, of), (of, Engineering), (Engineeri...","[(Director, of, Engineering), (of, Engineering..."
20339,Legal,Privacy,Director Of Reimbursement & HIPAA Privacy Officer,"[Director, Of, Reimbursement, &, HIPAA, Privac...","[(Director, Of), (Of, Reimbursement), (Reimbur...","[(Director, Of, Reimbursement), (Of, Reimburse..."
20340,Legal,Privacy,President and Senior Business Healthcare IT...,"[President, and, Senior, Business, Healthcare,...","[(President, and), (and, Senior), (Senior, Bus...","[(President, and, Senior), (and, Senior, Busin..."
...,...,...,...,...,...,...
20485,Legal,Privacy,Head of Legal consumer privacy and media,"[Head, of, Legal, consumer, privacy, and, media]","[(Head, of), (of, Legal), (Legal, consumer), (...","[(Head, of, Legal), (of, Legal, consumer), (Le..."
20486,Legal,Privacy,Privacy Counsel,"[Privacy, Counsel]","[(Privacy, Counsel)]",[]
20487,Legal,Privacy,Senior Governance & Privacy Consultant Busin...,"[Senior, Governance, &, Privacy, Consultant, B...","[(Senior, Governance), (Governance, &), (&, Pr...","[(Senior, Governance, &), (Governance, &, Priv..."
20488,Legal,Privacy,Principal Consultant internal CISO & Manager ...,"[Principal, Consultant, internal, CISO, &, Man...","[(Principal, Consultant), (Consultant, interna...","[(Principal, Consultant, internal), (Consultan..."


In [92]:
un_dict_fin={}
bi_dict_fin={}
tri_dict_fin={}

for i in list_fn_groups:
    temp=fn_group.get_group(i)
    list_div_groups=temp['Apollo Division'].unique().tolist()
    for j in list_div_groups:
        list_Temp=temp.loc[temp['Apollo Division']==j]
        un_dict_fin[i+"-"+j],bi_dict_fin[i+"-"+j],tri_dict_fin[i+"-"+j]=get_dicts(list_Temp)

In [94]:
un_dict_fin['Legal-Privacy']

{'Privacy': 141,
 'and': 57,
 '&': 54,
 'Data': 47,
 'Officer': 37,
 'Compliance': 27,
 'Security': 26,
 'Manager': 25,
 '-': 23,
 'Chief': 22,
 'of': 20,
 'Information': 20,
 'Director': 19,
 'Senior': 19,
 'Counsel': 17,
 'Head': 14,
 'Consultant': 12,
 'Global': 11,
 'Cybersecurity': 11,
 'Legal': 10,
 'Cyber': 9,
 'Protection': 9,
 'Associate': 9,
 'Governance': 8,
 'Risk': 8,
 'Product': 7,
 'Management': 7,
 'General': 7,
 'Lead': 7,
 'Marketing': 6,
 'Partner': 6,
 'Analyst': 6,
 'IT': 5,
 'Managing': 5,
 'Specialist': 5,
 'Sr.': 5,
 'DPO': 5,
 'President': 4,
 'Group': 4,
 'Digital': 4,
 'Program': 3,
 'Trust': 3,
 'Of': 3,
 'GRC': 3,
 'Vice': 3,
 'CISO': 3,
 'Principal': 3,
 'Strategy': 3,
 'Technology': 3,
 'security': 3,
 'Planning': 3,
 'privacy': 3,
 'Staff': 3,
 'University': 3,
 'Office': 3,
 'Advisory': 3,
 'Innovation': 3,
 'the': 3,
 'Health': 3,
 'Safety': 2,
 'Business': 2,
 'APAC': 2,
 'Regulatory': 2,
 'GDPR': 2,
 'Customer': 2,
 'Co-founder': 2,
 'Services': 2,
 

In [95]:
bi_dict_fin['Legal-Privacy']

{'Privacy Officer': 29,
 '& Privacy': 27,
 'Data Privacy': 26,
 'and Privacy': 19,
 'Privacy and': 17,
 'Chief Privacy': 13,
 'Privacy &': 13,
 'Senior Manager': 10,
 'Head of': 9,
 'Data Protection': 8,
 'Compliance and': 8,
 'and Data': 7,
 'Cybersecurity &': 7,
 'Information Security': 7,
 'General Counsel': 7,
 '& Data': 7,
 'Security &': 6,
 'Privacy Consultant': 6,
 'Compliance &': 6,
 'and Compliance': 6,
 'Governance &': 5,
 'Counsel and': 5,
 'and Information': 5,
 'Privacy Lead': 5,
 'Officer DPO': 5,
 'Director of': 4,
 'of Data': 4,
 'Security and': 4,
 'Protection Officer': 4,
 'and Security': 4,
 'Privacy Counsel': 4,
 'Manager -': 4,
 'Global Data': 4,
 'Data Governance': 4,
 'Security Compliance': 4,
 'Associate General': 4,
 '& Chief': 4,
 'Program Manager': 3,
 'Security Privacy': 3,
 'Vice President': 3,
 'Counsel -': 3,
 'Strategy and': 3,
 'Head Data': 3,
 'Data &': 3,
 'Chief Information': 3,
 '& Compliance': 3,
 'Privacy Specialist': 3,
 'University Legal': 3,
 '

In [96]:
tri_dict_fin['Legal-Privacy']

{'Chief Privacy Officer': 11,
 'Data Privacy and': 7,
 'Cybersecurity & Privacy': 7,
 'and Privacy Officer': 7,
 'Compliance and Privacy': 7,
 'Privacy & Data': 6,
 'Governance & Privacy': 5,
 'Compliance & Privacy': 5,
 'Privacy and Compliance': 5,
 'Security & Privacy': 4,
 'Security and Privacy': 4,
 'Data Protection Officer': 4,
 'and Data Privacy': 4,
 'Global Data Privacy': 4,
 'Security Compliance and': 4,
 '& Data Protection': 4,
 'Associate General Counsel': 4,
 'Head of Data': 3,
 'Data Privacy Consultant': 3,
 '& Privacy Officer': 3,
 'Data & Privacy': 3,
 'Privacy & Compliance': 3,
 '& Privacy Lead': 3,
 'Senior Manager -': 3,
 'Privacy and Security': 3,
 'University Legal Counsel': 3,
 'Legal Counsel and': 3,
 'Counsel and Information': 3,
 'and Information Privacy': 3,
 'Information Privacy Officer': 3,
 'and Privacy Analyst': 3,
 'General Counsel &': 3,
 '& Chief Privacy': 3,
 'Manager Data Privacy': 3,
 'Privacy Program Manager': 2,
 '& Privacy Consultant': 2,
 'of Data